In [1]:
using DynamicalSystems
using LinearAlgebra
using ProgressMeter
using Distributed
using Plots
using SharedArrays
using Base.Iterators
using DifferentialEquations

σ(x) = 1/(1+exp(-x));
W(α,θ) = α*[cos(θ) -sin(θ); sin(θ) cos(θ)];

function RNN(dx, x, p, n)
    α,θ = p
    dx.=tanh.(W(α,θ)*x).-x
    return
end
# Jacobian:
function RNN_jac(J, x, p, n)
    α,θ = p
    dx=tanh.(W(α,θ)*x)
    J.= Diagonal(1. .-dx.^2)*W(α,θ) - I
    return
end

αs = collect(1.1:0.1:10.)
αs = prepend!(αs, 1. .+[1e-3,1e-2])
αs = prepend!(αs, collect(0:0.1:1.))

θs = range(0, stop=π, length=100)

stuff_to_loop_over = product(αs,θs)|>collect;

In [5]:
using NPZ

In [2]:
Threads.nthreads()

14

In [3]:
numel = length(stuff_to_loop_over);
Λ₁s = zeros(Float64, length(αs)*length(θs));
Λ₂s = zeros(Float64, length(αs)*length(θs));
tmax = 2000.
transient = 500.

p = Progress(numel);
jj = Threads.Atomic{Int}(0)
l = Threads.SpinLock()

Threads.@threads for i in 1:numel
    
    α, θ = stuff_to_loop_over[i]
    ds = ContinuousDynamicalSystem(RNN, [0.5,0.5], [α θ], RNN_jac)
    λs = lyapunovs(ds, 1e4, Ttr = transient, dt=.1, alg=Tsit5(),tstops=0.:1e-2:tmax)
    Λ₁s[i]= λs[1]
    Λ₂s[i]= λs[2]
    
    Threads.atomic_add!(jj, 1)
    Threads.lock(l)
        update!(p, jj[])
    Threads.unlock(l)
end

Progress: 100%|█████████████████████████████████████████| Time: 1:20:046:02


In [6]:
npzwrite("LE_ode_tanh.npz", Dict("alpha" => αs, "thetas" => θs, "LE1" => Λ₁s, "LE2"=>Λ₂s))

In [ ]:
function GRU!(du,u,p,t)
    α,θ,β = p
    r = 0.5
    du .= (σ(β)-1).*(u - tanh.(0.5.*W(α,θ)*u))
end

In [ ]:
αs = range(2, stop=10, length=100+1)
θs = range(0, stop=π, length=100+1)
gates = range(-5, stop=5, length=2)
stuff_to_loop_over = product(αs,θs,gates)|>collect;

numel = length(stuff_to_loop_over);
Λ₁s = zeros(Float64, numel);
Λ₂s = zeros(Float64, numel);

p = Progress(numel);
jj = Threads.Atomic{Int}(0)
l = Threads.SpinLock()

Threads.@threads for i in 1:numel
    
    α, θ, gate = stuff_to_loop_over[i]
    ds = ContinuousDynamicalSystem(GRU!, [0.5,0.5], [α θ gate])
    λs = lyapunovs(ds, 1e4, Ttr = transient, dt=.1, alg=Tsit5(),tstops=0.:1e-2:tmax)
    Λ₁s[i]= λs[1]
    Λ₂s[i]= λs[2]
    
    Threads.atomic_add!(jj, 1)
    Threads.lock(l)
        update!(p, jj[])
    Threads.unlock(l)
end
npzwrite("LE_ode_gru.npz", Dict("alpha" => αs, "thetas" => θs, "gates"=>gates, "LE1" => Λ₁s, "LE2"=>Λ₂s))